In [44]:
import unittest
import os
from sqlitesec import SqliteSec

class TestSqliteSec(unittest.TestCase):
    def setUp(self):
        # Use a temporary database file for testing
        self.db_path = 'test.db'

        if os.path.exists(self.db_path):
            os.remove(self.db_path)
        self.key = b'test_key_1234567890123456'  # 16-byte key for testing
        self.sqlitesec = SqliteSec(self.key)


    def test_connection(self):
        """Test if the database connection is established and closed properly."""
        conn = self.sqlitesec.connect(self.db_path)
        self.assertIsNotNone(conn)
        self.sqlitesec.close(conn, self.db_path)
        # Check if file exists after closing (should be encrypted)
        self.assertTrue(os.path.exists(self.db_path))

    def test_encryption_decryption(self):
        """Test if data is correctly encrypted and decrypted."""
        conn = self.sqlitesec.connect(self.db_path)
        cursor = conn.cursor()
        # Create a table and insert data
        cursor.execute('CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, data TEXT)')
        test_data = 'Hello, world!'
        cursor.execute('INSERT INTO test (data) VALUES (?)', (test_data,))
        conn.commit()
        self.sqlitesec.close(conn, self.db_path)
        
        # Reconnect and check data
        conn = self.sqlitesec.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('SELECT data FROM test WHERE id=1')
        fetched_data = cursor.fetchone()[0]
        print("Data fetched from database:", fetched_data)
        self.assertEqual(fetched_data, test_data)
        self.sqlitesec.close(conn, self.db_path)


  
    def tearDown(self):
        # Remove the test database file if it exists
        if os.path.exists(self.db_path):
            os.remove(self.db_path)

# To run the tests in a Jupyter notebook, use the following:
unittest.main(argv=[''], verbosity=2, exit=False)



test_connection (__main__.TestSqliteSec)
Test if the database connection is established and closed properly. ... ok
test_encryption_decryption (__main__.TestSqliteSec)
Test if data is correctly encrypted and decrypted. ... 

Data fetched from database: Hello, world!


ok

----------------------------------------------------------------------
Ran 2 tests in 0.162s

OK


In [45]:
# Usage example
if os.path.exists("test.db"):
    os.remove("test.db")

key = b'blabla'
sqs = SqliteSec(key)

conn = sqs.connect("test.db")
cursor = conn.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS test (id INTEGER PRIMARY KEY, data TEXT)')
cursor.execute('INSERT INTO test (data) VALUES (?)', ('Hei, verden!',))
conn.commit()
sqs.close(conn, "test.db")


# Step 2: Read the data back

conn = sqs.connect("test.db")
cursor = conn.cursor()
cursor.execute('SELECT data FROM test WHERE id=1')
fetched_data = cursor.fetchone()[0]
print(fetched_data)
sqs.close(conn, "test.db")


Hei, verden!
